#### Import Env and check first

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

#### Works fine

In [2]:
from deep_research.prompts import clarify_with_user_instructions
print(clarify_with_user_instructions)


These are the messages that have been exchanged so far from the user asking for the report:
<Messages>
{messages}
</Messages>

Today's date is {date}.

Assess whether you need to ask a clarifying question, or if the user has already provided enough information for you to start research.
IMPORTANT: If you can see in the messages history that you have already asked a clarifying question, you almost always do not need to ask another one. Only ask another question if ABSOLUTELY NECESSARY.

If there are acronyms, abbreviations, or unknown terms, ask the user to clarify.
If you need to ask a question, follow these guidelines:
- Be concise while gathering all necessary information
- Make sure to gather all the information needed to carry out the research task in a concise, well-structured manner.
- Use bullet points or numbered lists if appropriate for clarity. Make sure that this uses markdown formatting and will be rendered correctly if the string output is passed to a markdown renderer.
-

## Build State for Agent

In [3]:
%%writefile ../src/deep_research/state_scope.py

from langgraph.graph import MessagesState
from typing_extensions import Optional, Annotated, Sequence
import operator
from langgraph.graph.message import BaseMessage,add_messages
from pydantic import BaseModel, Field

class AgentInputState(MessagesState):
    """Input State for the Agent. Only contains the user's input message"""
    pass

class AgentState(MessagesState):
    """State of the Agent while Agent researches on the topic"""
    research_brief : Optional[str]
    supervisor_messages : Annotated[Sequence[BaseMessage], add_messages]
    raw_notes: Annotated[list[str], operator.add] = []
    notes: Annotated[list[str], operator.add] = []
    final_report : str

class ClarifyWithUser(BaseModel):
    """Clarification Schema for clarifying before starting the researching"""
    need_clarification : bool = Field(
        description="Wether the user needs to be asked for clarifying question"
    )

    question : str = Field(
        description= "A question to ask the user to clarify the research scope"
    )

    verification : str = Field(
        description="Verify message that we will start research, after the user has provided the necessary information"
    )

class ResearchQuestion(BaseModel):
    """The Actual Search question to be provided for research"""
    research_brief : str = Field(
        description="A research question that will be used to guide the research"
    )
    

Overwriting ../src/deep_research/state_scope.py


Agent State defined now continue with the clarification node

#### Now create a open router wrapper for creating our chat models, this will help us explore opensource models and other paid models from openrouter

In [13]:
%%writefile ../src/deep_research/openrouter.py
"""
Module to init chat models from openrouter and use various opensource
and free models for deep research
"""

from langchain_openai import ChatOpenAI
from typing_extensions import Optional
import os



def init_chat_model(model: str, api_key: Optional[str], temperature: str=0, base_url: str = 'https://openrouter.ai/api/v1'):
    """Chat model initialization similar to langchain init_chat_model, but specific to openrouter"""
    open_router_key = api_key
    if not open_router_key and not os.getenv('OPENROUTER_API_KEY'):
        raise ValueError('API Key not provided, either provide OPENROUTER_API_KEY as evironment variable or provide in the function')        
    return ChatOpenAI(
        model = model,
        temperature = temperature,
        api_key = open_router_key if open_router_key else os.getenv('OPENROUTER_API_KEY'),
        base_url= base_url
    )

Overwriting ../src/deep_research/openrouter.py


### Making the clarification node

In [14]:
%%writefile ../src/deep_research/scope_research.py

"""
Contains Nodes and the graph flow to clarify with user and finally have determine the proper research scope
"""

from deep_research.state_scope import AgentState, ClarifyWithUser, ResearchQuestion, AgentInputState
from typing import Literal
from langgraph.types import Command
import os
from deep_research.prompts import clarify_with_user_instructions, transform_messages_into_research_topic_prompt
from langchain_core.messages import HumanMessage, get_buffer_string, AIMessage
from datetime import datetime
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from deep_research.openrouter import init_chat_model

load_dotenv()

def get_today_str():
    """return todays date in windows, different method for other os"""
    return datetime.now().strftime("%Y -%m -%d")

model = init_chat_model(model = "x-ai/grok-4-fast:free", api_key=os.getenv('OPENAI_API_KEY'), temperature=0)

def clarify_with_user(state : AgentState)-> Command[Literal["write_research_brief", "__end__"]]:
    """
    Determine if the user's request contains sufficient information to proceed with research.
    
    Uses structured output to make deterministic decisions and avoid hallucination.
    Routes to either research brief generation or ends with a clarification question.
    """

    structured_model = model.with_structured_output(ClarifyWithUser)
    response = structured_model.invoke([
        HumanMessage(content = clarify_with_user_instructions.format(
            messages = get_buffer_string(messages=state['messages']),
            date = get_today_str()
        ))
    ])

    if response.need_clarification:
        return Command(
            goto=END,
            update={"messages" : AIMessage(content=response.question)}
        )
    else:
        return Command(
            goto="write_research_brief",
            update={"messages" : AIMessage(content=response.verification)}
        )

def write_research_brief(state : AgentState):
    """
    Transform the conversation history into a comprehensive research brief.
    
    Uses structured output to ensure the brief follows the required format
    and contains all necessary details for effective research.
    """
    structured_model = model.with_structured_output(ResearchQuestion)

    response = structured_model.invoke([
        HumanMessage(content=transform_messages_into_research_topic_prompt.format(
            messages=get_buffer_string(messages=state.get('messages',[])),
            date=get_today_str()
        ))
    ])

    return {
        "research_brief" : response.research_brief,
        "supervisor_brief" : response.research_brief
    }


deep_research_builder = StateGraph(AgentState, input_schema = AgentInputState)

deep_research_builder.add_node('clarify_with_user', clarify_with_user)
deep_research_builder.add_node('write_research_brief', write_research_brief)

deep_research_builder.add_edge(START, 'clarify_with_user')
deep_research_builder.add_edge('write_research_brief', END)

scope_research = deep_research_builder.compile()


Overwriting ../src/deep_research/scope_research.py


In [5]:
# from IPython.display import Image, display
# print(scope_research.get_graph(xray=True).draw_mermaid())

### Test

In [15]:
from deep_research.scope_research import deep_research_builder
from langgraph.checkpoint.memory import InMemorySaver
from langchain_core.messages import HumanMessage

checkpointer = InMemorySaver()
scope = deep_research_builder.compile(checkpointer = checkpointer)
thread = {"configurable": {"thread_id": "1"}}
result = scope.invoke({"messages": [HumanMessage(content="I want to research on some attraction games")]}, config=thread)

In [16]:
print(result)

{'messages': [HumanMessage(content='I want to research on some attraction games', additional_kwargs={}, response_metadata={}, id='49efe751-b1a2-4af9-86dc-5c891c861789'), AIMessage(content="Could you please clarify what you mean by 'attraction games'? For example, are you referring to amusement park attractions, video games with attraction themes, social or dating games, or something else? Also, what specific aspects would you like to research, such as history, popular examples, or trends?", additional_kwargs={}, response_metadata={}, id='ddde7572-bf89-448f-b6e9-bec84592fe09')], 'supervisor_messages': [], 'raw_notes': [], 'notes': []}


In [18]:
result = scope.invoke({"messages" : [HumanMessage(content="Lets research on the indoor attraction arenas like lasertag, bowling and other such kinds of game")]}, config=thread)

In [19]:
print(result)

{'messages': [HumanMessage(content='I want to research on some attraction games', additional_kwargs={}, response_metadata={}, id='49efe751-b1a2-4af9-86dc-5c891c861789'), AIMessage(content="Could you please clarify what you mean by 'attraction games'? For example, are you referring to amusement park attractions, video games with attraction themes, social or dating games, or something else? Also, what specific aspects would you like to research, such as history, popular examples, or trends?", additional_kwargs={}, response_metadata={}, id='ddde7572-bf89-448f-b6e9-bec84592fe09'), HumanMessage(content='Lets research on the indoor attraction arenas like lasertag, bowling and other such kinds of game', additional_kwargs={}, response_metadata={}, id='ab27630f-2fca-473c-88e0-e7bc76eb2546'), AIMessage(content='Thank you for the clarification. I understand you want to research indoor attraction arenas, such as laser tag, bowling, and other similar games. I now have sufficient information to proc

In [22]:
from utils import format_messages
format_messages(result['messages'])
print(result['research_brief'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ I want to research on some attraction games                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Could you please clarify what you mean by 'attraction games'? For example, are you referring to amusement park  │
│ attractions, video games with attraction themes, social or dating games, or something else? Also, what specific │
│ aspects would you like to research, such as history, popular examples, or trends?                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Lets research on the indoor attraction arenas like lasertag, bowling and other such kinds of game               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Thank you for the clarification. I understand you want to research indoor attraction arenas, such as laser tag, │
│ bowling, and other similar games. I now have sufficient information to proceed with the research.               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

I want to research indoor attraction arenas, such as laser tag, bowling, and other similar kinds of games, including their features, popular examples, operational aspects, and trends, while considering unspecified dimensions like locations, costs, or target audiences as open and flexible without assumptions.
